In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import math
from sympy import symbols, Eq, init_printing

filters= [["U", "violet"], ["B", "blue"], ["V", "green"], ["R", "red"], ["I", "#800000"]]

H0=65

init_printing()
M, delta_m_15, delta_m2_15, logH0 = symbols('M_{max}(V) Δm_{15}(B) Δm²_{15}(B) log_{10}(H_{0})')
phillips_eq = Eq(M, ( 0.736 * delta_m_15 + 0.182 * delta_m2_15 ) + 5 * logH0 - 19.504 )


for file in glob.glob("Supernova_all/cfalc_allsn/*_UBVRI.dat"):
    try:
        # read the data file into a dataframe
        df = pd.read_table(file, comment="#", sep="[ \t]+", engine="python", header=0, names=["HJD", "U", "Uerr", "B", "Berr", "V", "Verr", "R", "Rerr", "I", "Ierr"])

        # compute, if 10+ observations are available
        if len(df) >= 10:
            # remove missing data
            df[df == 99.999] = None
            df[df == 99.99] = None
            df[df == 9.999] = None

            # convert HJD to days from first observation
            df["HJD"]-= df["HJD"].iat[0]
            
            # correct mag to -mag
            for filter, color in filters:
                df[filter]= -df[filter]
            
            # plot light curves for each filter
            for filter, color in filters:
                plt.errorbar(df["HJD"].values, df[filter].values, label=filter, color=color, xerr=df[filter+"err"].values, capsize=4)

            max_index= df["B"].idxmax()
            max_day=df["HJD"].iat[max_index]
            max_day15=max_day+15
            closest_index=(df["HJD"]-max_day15).abs().idxmin()
            closest_day=df["HJD"].iat[closest_index]
            plt.axvline(x=max_day, color='yellow', linestyle='--', linewidth=1, label='Bmax')
            plt.axvline(x=max_day15, color='grey', linestyle='dotted', linewidth=1, label='Bmax+15')
            plt.axvline(x=closest_day, color='black', linestyle='--', linewidth=1, label='closest')
            
            plt.title(file)
            plt.xlabel("days since first observation")
            plt.ylabel("mag")
            plt.legend()
            
            # show the plot
            plt.show()

            # print our findings
            max_mag=df["B"].iat[max_index]
            day15_mag=df["B"].iat[closest_index]
            delta_mag=day15_mag-max_mag
            print(f"Bmax @ day {max_day:.2f} with mag {max_mag}")
            print(f"Bmax+15 @ day {closest_day:.2f} with mag {day15_mag}")
            print(f"Δm= {delta_mag:.2f}")

            display(phillips_eq)
#            print(pretty(phillips_eq, use_unicode=True))
            
            M_max= -19.504 + 0.736*delta_mag + 0.182*delta_mag*delta_mag + 5*math.log10(H0)
            d=10**((max_mag - M_max + 5) / 5)
            print(f"Mmax(V)= {M_max:.2f}")
            print(f"d= {d:.2f} pc")
            
    except Exception as oops:
        print(f"{file} ignored due to error: {oops}")

    print()
    print()
    